In [ ]:
#!nvidia-smi                       # uncomment to check if you have GPU access

import os

HOME = os.getcwd()
print(HOME)

# Pip install ultralytics

!pip install ultralytics==8.0.20


# gradio for demonstrations
!pip install -q gradio
import gradio as gr

from IPython import display
display.clear_output()

#import ultralytics
#ultralytics.checks()

from ultralytics import YOLO

#from IPython.display import display, Image

import shutil
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image as img
import cv2
from matplotlib import pyplot as plt
#from google.colab.patches import cv2_imshow
import re
import yaml



In [ ]:
## Download the Data to google colab if you do not upload it locally

## The model file " .pt" must be manually uploaded

!mkdir {HOME}/datasets
%cd {HOME}/datasets


!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="xXZVu6Pm2uClbW6QJTmx")
project = rf.workspace("container").project("containerletters")
dataset = project.version(2).download("yolov8")

/content/datasets
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to containerletters-2 in yolov8:: 100%|██████████| 4440/4440 [00:01<00:00, 2913.30it/s]


In [ ]:
##function to rename pictures(dataset pictures have very long names) to make it easier to work with

def rename_pictures(folder_path):
    i = 1
    for filename in os.listdir(folder_path):
        #if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            new_name = "test" + str(i) + os.path.splitext(filename)[1]
            os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_name))
            i += 1


In [ ]:
#!unrar x "/content/containerTest.rar" "/content/datasets"

In [ ]:
import shutil
shutil.make_archive('labels50', 'zip', '/content/runs/detect/predict4/labels')

In [ ]:

folder_path="/content/datasets/containerletters-2/test/images"
folder_path_label="/content/datasets/containerletters-2/test/labels"

# (Optional) 
#rename_pictures(folder_path)



In [ ]:
#on pictures/videos thats uploaded to Google colab, or any other source if run locally

%cd {HOME}
!yolo task=detect mode=predict model="/content/bestChar3.pt" conf=0.30 save_txt=True hide_conf=True line_thickness=1 save=True source="/content/datasets/containerletters-2/test/images"

In [ ]:

#from PIL import Image

def get_string_txt(folder_path, text_folder_path, processing_function):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            text_file_path = os.path.join(text_folder_path, os.path.splitext(filename)[0]+ ".txt")
            image = img.open(image_path)
            processed_data = processing_function(text_file_path)
            text_path = os.path.splitext(image_path)[0] + ".txt"
            text_path = str(text_path)
            with open(text_path, "w") as text_file:
                text_file.write(str(processed_data))


In [ ]:
#go from yolo relative bounding box to actual bounding box

def relative_to_fixed(bounding_boxes_file, image ):

    dh, dw, _ = image.shape
    class_id, x_center, y_center, w, h = bounding_boxes_file
    x_center, y_center, w, h = float(x_center), float(y_center), float(w), float(h)
    x_center = round(x_center * dw)
    y_center = round(y_center * dh)
    w = round(w * dw)
    h = round(h * dh)
#    x_center = (x_center * dw)
#    y_center = (y_center * dh)
#    w = (w * dw)
#    h = (h * dh)
  #  x = round(x_center - w / 2)
    x = round(x_center)
    y = round(y_center - h / 2)
   # y = round(y_center)
    return (class_id, x, y, w, h)




In [ ]:
#combining the bounding boxes to go from chars -> words

def combine_bounding_boxes(bounding_boxes_file):
    min_x = float("inf")
    min_y = float("inf")
    max_x = 0
    max_y = 0
    labels = []
    bounding_boxes = []
    yaml_file ="/content/datasets/containerletters-2/data.yaml"
#    with open(yaml_file, 'r') as f:
#         class_mapping = yaml.load(f, Loader=yaml.FullLoader)
    with open('/content/datasets/containerletters-2/data.yaml') as f:
         class_mapping = yaml.safe_load(f)
  #       print(class_mapping.items())
  #       print(list(class_mapping))
    with open(bounding_boxes_file, 'r') as f:
        for line in f:
            label, x, y, w, h = line.strip().split()
            x1, y1, x2, y2 = float(x), float(y), float(x) + float(w), float(y) + float(h)
            bounding_boxes.append((x1, y1, x2, y2, label))
    # sort the bounding boxes based on their x and y coordinates
    bounding_boxes.sort(key=lambda x: (x[0], x[1]))   # (x[0], x[1])) for horizontal, switched for vertical
    for box in bounding_boxes:
    #    print(box[4])
    #    print(type(box[4]))
    #    labels.append(class_mapping.get(int(box[4])))
        labels.append(class_mapping["names"][int(box[4])])
    #    print(labels)
        min_x = min(min_x, box[0])
        min_y = min(min_y, box[1])
        max_x = max(max_x, box[2])
        max_y = max(max_y, box[3])
    combined_label = "".join(labels)
    return (combined_label, min_x, min_y, max_x-min_x, max_y-min_y)


In [ ]:
#make String from model prediction

picture_path = "/content/runs/detect/predict3"
label_path="/content/runs/detect/predict3/labels"

get_string_txt(picture_path, label_path, combine_bounding_boxes)

In [ ]:
#make String from true label

picture_path = "/content/datasets/containerletters-2/test/images"
label_path="/content/datasets/containerletters-2/test/labels"

get_string_txt(picture_path, label_path, combine_bounding_boxes)

In [ ]:
import os
import Levenshtein

def compare_folders4(folder1, folder2):
    files1 = [os.path.join(folder1, f) for f in os.listdir(folder1) if os.path.isfile(os.path.join(folder1, f))]
    files2 = [os.path.join(folder2, f) for f in os.listdir(folder2) if os.path.isfile(os.path.join(folder2, f))]
    
    text_files1 = [f for f in files1 if f.endswith('.txt')]
    text_files2 = [f for f in files2 if f.endswith('.txt')]
    
    num_files = min(len(text_files1), len(text_files2))
    
    identical_count = 0
    non_identical_files = []
    non_identical_strings1 = []
    non_identical_strings2 = []
    closest_strings1 = []
    
    # Create a list of all the strings in folder1
    folder1_strings = []
    for f in text_files1:
        with open(f, 'r') as file:
            string1 = file.readline().split(',')[0]
            folder1_strings.append(string1)
    
    for i in range(num_files):
        with open(text_files1[i], 'r') as f1, open(text_files2[i], 'r') as f2:
            string1 = f1.readline().split(',')[0]
            string2 = f2.readline().split(',')[0]
            if string1 == string2:
                identical_count += 1
            else:
                non_identical_files.append(os.path.basename(text_files2[i]))
                non_identical_strings1.append(string1)
                non_identical_strings2.append(string2)
                closest_string1 = min(folder1_strings, key=lambda x: Levenshtein.distance(string2, x))
                closest_strings1.append(closest_string1)
    
    # Calculate accuracy using closest matches from folder1
    num_files2 = len(non_identical_files)
    identical_count2 = 0
    for i in range(num_files2):
        if closest_strings1[i] == non_identical_strings1[i]:
            identical_count2 += 1
    #unprocessed accuracy
    accuracy = identical_count / num_files * 100 if num_files != 0 else 0
    #accuracy of Levenshtein closest string on wrong labels 
    accuracy2 = identical_count2 / num_files2 * 100 if num_files2 != 0 else 0
    #total accuracy after some Levenshtein fixes
    accuracy3 = (identical_count + identical_count2) / num_files * 100 if num_files2 != 0 else 0
    
    
    print(f"Number of text files compared: {num_files}")
    print(f"Number of non-identical files: {len(non_identical_files)}")
    print(f"Number of non-fixable files:   {len(non_identical_files) - identical_count2}")
    print(f"Accuracy with no correction:   {accuracy:.2f}%")
    print(f"Correcteable percentage:       {accuracy2:.2f}%")
    print()
    print("----------------------------------------------------")
    print(f"total Accuracy after fixes:    {accuracy3:.2f}%")
    print("----------------------------------------------------")
    print()
    if len(non_identical_files) > 0:
        print("Non-identical files:")
        for i, file in enumerate(non_identical_files):
            print(f"{i+1}. {file}")
            print(f"   True String:      {non_identical_strings1[i]}")
            print(f"   Predicted String: {non_identical_strings2[i]}")
            print(f"   Closest match:    {closest_strings1[i]}")


In [ ]:


true_label = "/content/datasets/containerletters-2/test/images"
predicted_label = "/content/runs/detect/predict"

print("-------bestChar45 model-------")
accuracy = compare_folders4(true_label, predicted_label)


-------bestChar45 model-------
Number of text files compared: 216
Number of non-identical files: 47
Number of non-fixable files:   4
Accuracy with no correction:   78.24%
Correcteable percentage:       91.49%

----------------------------------------------------
total Accuracy after fixes:    98.15%
----------------------------------------------------

Non-identical files:
1. IMG_20210821-17001398_jpg.rf.1981345f575889237b6211687f223fae.txt
   True String:      ('MRKU8347643'
   Predicted String: ('MRRKU8347643'
   Closest match:    ('MRKU8347643'
2. 580_jpg.rf.9b887ac7dc355286949935ce2836f41f.txt
   True String:      ('MNBU0188648'
   Predicted String: ('MNBU9188848'
   Closest match:    ('MNBU0188648'
3. email8_jpg.rf.f661f16ece8eef0fcb9f37cb26e9246c.txt
   True String:      ('LNXU8456131'
   Predicted String: ('ZXU8456131'
   Closest match:    ('LNXU8456131'
4. IMG_20210911-23133270_jpg.rf.600fa39234142e3aa3fcf02eb632f23b.txt
   True String:      ('MSKU9985351'
   Predicted String: 

In [ ]:


true_label = "/content/datasets/containerletters-2/test/images"
predicted_label = "/content/runs/detect/predict2"

print("-------bestChar4 model-------")
accuracy = compare_folders4(true_label, predicted_label)


-------bestChar4 model-------
Number of text files compared: 216
Number of non-identical files: 58
Number of non-fixable files:   1
Accuracy with no correction:   73.15%
Correcteable percentage:       98.28%

----------------------------------------------------
total Accuracy after fixes:    99.54%
----------------------------------------------------

Non-identical files:
1. IMG_20210828-13225458_jpg.rf.d2408c4186c61bb1d2bc435e990020b9.txt
   True String:      ('APHU6047613'
   Predicted String: ('APIU6047613'
   Closest match:    ('APHU6047613'
2. IMG_20210912-14430889_jpg.rf.9332451825d1c18bcd2c3355dad3f4c9.txt
   True String:      ('TRIU8061129'
   Predicted String: ('TRU8061129'
   Closest match:    ('TRIU8061129'
3. IMG_20210821-17001398_jpg.rf.1981345f575889237b6211687f223fae.txt
   True String:      ('MRKU8347643'
   Predicted String: ('MRRKU8347643'
   Closest match:    ('MRKU8347643'
4. 580_jpg.rf.9b887ac7dc355286949935ce2836f41f.txt
   True String:      ('MNBU0188648'
   Pred

In [ ]:


true_label = "/content/datasets/containerletters-2/test/images"
predicted_label = "/content/runs/detect/predict3"

print("-------bestChar3 model-------")
accuracy = compare_folders4(true_label, predicted_label)


-------bestChar3 model-------
Number of text files compared: 216
Number of non-identical files: 54
Number of non-fixable files:   1
Accuracy with no correction:   75.00%
Correcteable percentage:       98.15%

----------------------------------------------------
total Accuracy after fixes:    99.54%
----------------------------------------------------

Non-identical files:
1. IMG_20210912-14430889_jpg.rf.9332451825d1c18bcd2c3355dad3f4c9.txt
   True String:      ('TRIU8061129'
   Predicted String: ('TRU8061129'
   Closest match:    ('TRIU8061129'
2. 580_jpg.rf.9b887ac7dc355286949935ce2836f41f.txt
   True String:      ('MNBU0188648'
   Predicted String: ('MNBU18648'
   Closest match:    ('MNBU0188648'
3. email18_jpg.rf.e9841a88341231000b527b4ebfad2e73.txt
   True String:      ('CLHU8855811'
   Predicted String: ('CLU8855811'
   Closest match:    ('CLHU8855811'
4. IMG_20210908-21062224_jpg.rf.b2e013f5158a2ca861be847647696299.txt
   True String:      ('TGBU5412109'
   Predicted String: ('TG